In [2]:
import os
from dotenv import load_dotenv
import sys


In [3]:
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

In [4]:
sys.path.append("/home/lillian/Documents/TenAcademy/enterprise-rag-exercise/backend")

In [5]:
from retrival import file_reader
from retrival import test_contextualized_answer

In [6]:
import pinecone


pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY') or '3306f52a-a64a-46dd-b81a-0d073fb5a072',
    environment=os.environ.get('PINECONE_ENVIRONMENT') or 'gcp-starter'
)

ModuleNotFoundError: No module named 'pinecone'

In [7]:
import time

index_name = 'llama-2-rag'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

index = pinecone.Index(index_name)
    

NameError: name 'pinecone' is not defined

In [7]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.04838,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

In [8]:
context_message = file_reader("/home/lillian/Documents/TenAcademy/enterprise-rag-exercise/backend/context.txt")
document = context_message.split('\n')


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer()

In [12]:
context_vector = vectorizer.fit_transform(document)

In [30]:
print(context_vector)

  (0, 413)	0.7266547071407753
  (0, 99)	0.6870028650523621
  (1, 138)	0.10080123133878717
  (1, 583)	0.10855373183519164
  (1, 347)	0.07778124866569268
  (1, 10)	0.10080123133878717
  (1, 579)	0.10855373183519164
  (1, 672)	0.09530074470038043
  (1, 351)	0.10855373183519164
  (1, 193)	0.10855373183519164
  (1, 404)	0.10855373183519164
  (1, 236)	0.09530074470038043
  (1, 115)	0.10855373183519164
  (1, 183)	0.10855373183519164
  (1, 55)	0.17509648840795186
  (1, 15)	0.10855373183519164
  (1, 240)	0.10855373183519164
  (1, 168)	0.10855373183519164
  (1, 205)	0.10080123133878717
  (1, 421)	0.10855373183519164
  (1, 173)	0.10080123133878717
  (1, 219)	0.10855373183519164
  (1, 539)	0.10855373183519164
  (1, 450)	0.10855373183519164
  (1, 455)	0.10855373183519164
  :	:
  (207, 490)	0.2995873471089458
  (207, 385)	0.2995873471089458
  (207, 237)	0.2995873471089458
  (207, 225)	0.2995873471089458
  (207, 460)	0.2995873471089458
  (207, 522)	0.2630116606736323
  (207, 93)	0.27819194210615195
 

In [8]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,):
    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

    

In [9]:
import openai
import IPython

def get_completion(params, messages):
    """ GET completion from openai api"""

    response = openai.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response



In [11]:
#PROMPTING EXPERIMENT
#few-shot prompts
prompt = [{
  "role": "user" , 
  "content":"""The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""}]

response = openai.chat.completions.create(
model = "gpt-3.5-turbo",
messages = prompt,
temperature = 1.0,
max_tokens = 256,
top_p =1,
frequency_penalty = 0) 
IPython.display.Markdown(response.choices[0].message.content)

AttributeError: module 'openai' has no attribute 'chat'

In [18]:
params = set_open_params()
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.

In [19]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)


Step 1: You bought 10 apples.
Step 2: You gave 2 apples to the neighbor, so you have 10 - 2 = 8 apples remaining.
Step 3: You gave 2 more apples to the repairman, so you have 8 - 2 = 6 apples remaining.
Step 4: You bought 5 more apples, so you have 6 + 5 = 11 apples.
Step 5: You ate 1 apple, so you have 11 - 1 = 10 apples remaining.

Therefore, you have 10 apples remaining.

In [32]:
import weaviate
import os
from dotenv import load_dotenv

load_dotenv()

weaviate_api_key = os.getenv("WEAVIATE_API_kEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

client = weaviate.Client(
    url="https://precision-rag-ov8u665q.weaviate.network", 
    auth_client_secret=weaviate.AuthApiKey(api_key= weaviate_api_key),
     additional_headers={
        "X-OpenAI-Api-Key": openai_api_key  
    }
      )

client.schema.get()

{'classes': [{'class': 'Document',
   'description': 'Represents a document with content and vector representation.',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'generative-openai': {},
    'text2vec-openai': {'baseURL': 'https://api.openai.com',
     'model': 'ada',
     'modelVersion': '002',
     'type': 'text',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'The content of the document.',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'content',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': 'The category',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-

In [17]:
class_obj = {
    "class": "Document",
    "description" : "Represents a document with content and vector representation.",
     "properties": [
           {
               "name": "content",
               "dataType": ["text"],
               "description": "The content of the document."
           },
           {
               "name": "category",
               "dataType": ["text"],
               "description": "The category"
           }
     ],
    "vectorizer": "text2vec-openai",  
    "moduleConfig": {
        "text2vec-openai": {},
        "generative-openai": {} 
        
    }
}

client.schema.create_class(class_obj)



In [21]:
schema  = client.schema.get()
print(schema)

{'classes': [{'class': 'Document', 'description': 'Represents a document with content and vector representation.', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'moduleConfig': {'generative-openai': {}, 'text2vec-openai': {'baseURL': 'https://api.openai.com', 'model': 'ada', 'modelVersion': '002', 'type': 'text', 'vectorizeClassName': True}}, 'multiTenancyConfig': {'enabled': False}, 'properties': [{'dataType': ['text'], 'description': 'The content of the document.', 'indexFilterable': True, 'indexSearchable': True, 'moduleConfig': {'text2vec-openai': {'skip': False, 'vectorizePropertyName': False}}, 'name': 'content', 'tokenization': 'word'}, {'dataType': ['text'], 'description': 'The category', 'indexFilterable': True, 'indexSearchable': True, 'moduleConfig': {'text2vec-openai': {'skip': False, 'vectorizePropertyName': False}}, 'name': 'category', 'tokenization': 'word'}], 're

In [14]:
client.schema.delete_class("Document")

In [20]:
class_object = {
    
    "class": "Article",
    "vectorizer": "text2vec-cohere",
    "moduleConfig": {
        "text2vec-cohere": {
            "model": "embed-multilingual-v2.0",
        },
    },
    "properties": [
        {
            "name": "title",
            "dataType": ["text"]
        },
        {
            "name": "body",
            "dataType": ["text"]
        },
    ],
}
client.schema.create_class(class_object)

In [27]:
with open("context.txt", "r", encoding="utf-8") as file:
    document_content = file.read()


In [28]:
data_object = {
    "class": "Document", 
    "properties": {
        "text": document_content
    }
}


In [33]:
from weaviate.util import generate_uuid5

data_object["uuid"]= generate_uuid5(data_object)


In [34]:
client.data_object.create(class_name="Document",data_object= data_object)

'cb9aac85-1d85-4b0f-9a9c-f820cc76ff9c'

In [43]:
query_result = client.query.get(class_name="Document").do()

data_objects = query_result["data"]["Get"]["Document"]
for item in data_objects:
    text_property = item["text"]
    print(f"Text: {text_property}")

AttributeError: No 'properties' or 'additional properties' specified to be returned. At least one should be included.